<a href="https://colab.research.google.com/github/aneeshoberoi/churnhw/blob/main/ChurnNoteBook_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [164]:
import pandas as pd
import numpy as np
from collections import OrderedDict

#pd.set_option('display.float_format', lambda x: '%.0f' % x)
pd.options.display.float_format = "{:,.2f}".format
pd.set_option("display.precision",2)

In [165]:
#source='https://raw.githubusercontent.com/aneeshoberoi/churnhw/main/churn_homework%20-%20homework_orders.csv?token=AP7KM2ST5B3PDSL5PKLDBGC7SX4FG'
#raw=pd.read_csv(source, sep=",")

In [166]:
#raw=df.head(100)
#raw.set_index('order_id')

In [167]:
# assigning to working copy for indicator variables

# **Basic Statistics**

In [168]:
raw.customer.nunique()

10316

In [169]:
raw.restaurant_total.sum()

1372249.61

In [170]:
raw.order_id.nunique()

42198

In [171]:
raw.customer.isna().sum()

5

# ***Data Cleaning***

In [172]:
# assigning to working copy for indicator variables
test=raw

In [173]:
# Remove zero dollar txns
test=test[test['restaurant_total']>0]

In [174]:
# Remove rows with customer number = na
test=test.dropna(subset=['customer'])

In [175]:
test.customer.nunique()

10316

In [176]:
test.order_id.nunique()

41906

In [177]:
test.restaurant_total.sum()

1372113.6300000001

In [178]:
# Variable Conversion
test['date_purchased']=pd.to_datetime(test['date_purchased'])
test['dateonly_purchased']=test['date_purchased'].dt.date
test['houronly_purchased']=test['date_purchased'].dt.hour
test['datehour_purchased']=test['dateonly_purchased'].astype(str) + "-" + test['houronly_purchased'].astype(str)

test['restaurant_total']=test['restaurant_total'].astype(float)

In [179]:
# Removing Duplicate Txns - i.e. those made within the same hour at the same shop by a given customer. Keeping last txn only.
test=test.drop_duplicates(subset=['customer', 'shops_id', 'datehour_purchased'], keep='last')


In [180]:
test.customer.nunique()

10316

In [181]:
test.order_id.nunique()

39880

In [182]:
test.restaurant_total.sum()

1301395.08

# ***Data Prep - Indicators***

In [183]:
 #Enumerating Purchases
 test.sort_values(by=['customer','date_purchased'])

test = test.assign(order_enum=test.groupby('customer')['datehour_purchased'].cumcount()+1)
test['total_order']=test.groupby('customer')['order_id'].transform('count')
test['total_sales']=test.groupby('customer')['restaurant_total'].transform('sum')

In [184]:
test['bin_orders']=np.where(test['total_order']<7,test['order_enum'],7)

In [185]:
# Calculating Journey Data

test['time_between_purch']=test.groupby(['customer'])['date_purchased'].diff()
test['spend_growth']=test.groupby(['customer'])['restaurant_total'].diff()


# Preparing Zip-Cust Level Variables




In [186]:
#Assigning raw dataset to working copy for this section
shopszip=test[['postal_code', 'shops_id','customer','total_order']].set_index('postal_code')
shopszip=shopszip.drop_duplicates(subset=['customer', 'shops_id'], keep='last')

In [187]:
# calculating shops and customers per postal code
shopszip['shops_per_zip']=shopszip.groupby('postal_code').shops_id.nunique()
shopszip['cust_per_zip']=shopszip.groupby('postal_code').customer.nunique()
shopszip['shops_per_cust_per_zip']=shopszip['shops_per_zip']/shopszip['cust_per_zip']

In [188]:
# Of customers placing orders in a given zip, what # and  % have placed more than 2 orders in total?
temp=shopszip[shopszip['total_order']>1]
shopszip['zip_cust_2_more']=temp.groupby('postal_code')['customer'].count()
shopszip['zip_pct_cust_2_more']=shopszip['zip_cust_2_more']*100/shopszip['cust_per_zip']

# ***Preparing Shop-Cust Level Variables***

In [189]:
#Total customers served by each shop

In [190]:
shopscust=shopszip.set_index('shops_id')
shopscust=shopscust.drop(columns=['shops_per_zip','cust_per_zip','zip_cust_2_more', 'zip_pct_cust_2_more','shops_per_cust_per_zip'] )

# How many customers have placed at least one order at a given shop?

shopscust['cust_per_shop']=shopscust.groupby('shops_id').customer.nunique()

In [191]:
# Of customers placing orders at a given shop, what # and % have placed > 2 total orders/
t=shopscust[shopscust['total_order']>1]
shopscust['shop_cust_2_more']=t.groupby('shops_id')['customer'].count()
shopscust['shop_pct_cust_2_more']=shopscust['shop_cust_2_more']*100/shopscust['cust_per_shop']

In [192]:
shopscust.head(10)

,customer,total_order,cust_per_shop,shop_cust_2_more,shop_pct_cust_2_more
shops_id,,,,,
64392,"15,557,129,447.00",5,3,3.00,100.00
34911,"18,529,185,191.00",1,8,6.00,75.00
24807,"15,315,600,200.00",1,1,nan,nan
13989,"13,306,584,713.00",1,1,nan,nan
44211,"15,487,909,931.00",1,4,2.00,50.00
43299,"6,041,663,939.00",1,1,nan,nan
30378,"10,435,705,412.00",2,3,3.00,100.00
72708,"8,037,565,352.00",2,2,2.00,100.00
48780,"14,548,429,994.00",1,7,3.00,42.86


# ***Preparing Shop-Sales Level Variables***

In [193]:
shopsales=test[['shops_id','customer','order_id','restaurant_total']] 

In [194]:
shopsales['total_shop_sales']=shopsales.groupby('shops_id').restaurant_total.transform('sum')
shopsales['total_shop_orders']=shopsales.groupby('shops_id').order_id.transform('count')
shopsales['total_shop_cust_base']=shopsales.groupby('shops_id').customer.transform('nunique')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

In [195]:
shopsales=shopsales.drop_duplicates(subset=['shops_id'], keep='last').drop(columns=['customer','order_id','restaurant_total'])

In [196]:
shopsales['sales_qunitiles']=pd.qcut(shopsales['total_shop_sales'],5,labels=False)
shopsales['order_qunitiles']=pd.qcut(shopsales['total_shop_orders'].rank(method='first'),5,labels=False)

#pd.qcut(df['a'].rank(method='first'), 3, labels=False)

# ***Combining to create final analytical dataset***

In [197]:
shopscust_final=shopscust.reset_index()
shopscust_final=shopscust_final.drop_duplicates(subset=['shops_id'], keep='last').drop(columns=['customer','total_order']).set_index('shops_id')


In [198]:
shopscust_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6058 entries, 24807 to 18948
Data columns (total 3 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   cust_per_shop         6058 non-null   int64  
 1   shop_cust_2_more      5213 non-null   float64
 2   shop_pct_cust_2_more  5213 non-null   float64
dtypes: float64(2), int64(1)
memory usage: 189.3 KB


In [199]:
# Merging shopscust with original dataset

test2=test.merge(shopscust_final,left_on='shops_id',right_on='shops_id') 


In [200]:
test2[test2['customer']==6196987925]

,order_id,source,date_purchased,shipping_type,payment_method,promo_value,restaurant_total,customer,shops_id,state,postal_code,dateonly_purchased,houronly_purchased,datehour_purchased,order_enum,total_order,total_sales,bin_orders,time_between_purch,spend_growth,cust_per_shop,shop_cust_2_more,shop_pct_cust_2_more
314,47474511,slicelife,2019-03-13 16:33:51+00:00,Delivery,cash,0,20.92,"6,196,987,925.00",27246,WA,98101,2019-03-13,16.00,2019-03-13-16.0,1,23,463.31,7,NaT,nan,2,2.00,100.00
315,47513940,slicelife,2019-03-14 13:15:11+00:00,Delivery,cash,0,20.92,"6,196,987,925.00",27246,WA,98101,2019-03-14,13.00,2019-03-14-13.0,2,23,463.31,7,0 days 20:41:20,0.00,2,2.00,100.00
316,47566653,slicelife,2019-03-14 18:44:08+00:00,Delivery,cash,0,18.72,"6,196,987,925.00",27246,WA,98101,2019-03-14,18.00,2019-03-14-18.0,3,23,463.31,7,0 days 05:28:57,-2.20,2,2.00,100.00
317,47716284,slicelife,2019-03-16 14:25:01+00:00,Delivery,cash,0,20.92,"6,196,987,925.00",27246,WA,98101,2019-03-16,14.00,2019-03-16-14.0,4,23,463.31,7,1 days 19:40:53,2.20,2,2.00,100.00
318,47947731,slicelife,2019-03-19 17:56:56+00:00,Delivery,cash,0,23.12,"6,196,987,925.00",27246,WA,98101,2019-03-19,17.00,2019-03-19-17.0,5,23,463.31,7,3 days 03:31:55,2.20,2,2.00,100.00
319,47993268,slicelife,2019-03-20 16:09:30+00:00,Delivery,cash,0,23.12,"6,196,987,925.00",27246,WA,98101,2019-03-20,16.00,2019-03-20-16.0,6,23,463.31,7,0 days 22:12:34,0.00,2,2.00,100.00
320,48058374,slicelife,2019-03-21 15:29:23+00:00,Delivery,cash,0,19.82,"6,196,987,925.00",27246,WA,98101,2019-03-21,15.00,2019-03-21-15.0,7,23,463.31,7,0 days 23:19:53,-3.30,2,2.00,100.00
321,48162981,slicelife,2019-03-22 16:00:47+00:00,Delivery,cash,0,18.72,"6,196,987,925.00",27246,WA,98101,2019-03-22,16.00,2019-03-22-16.0,8,23,463.31,7,1 days 00:31:24,-1.10,2,2.00,100.00
322,48254211,slicelife,2019-03-23 15:15:08+00:00,Delivery,cash,0,22.02,"6,196,987,925.00",27246,WA,98101,2019-03-23,15.00,2019-03-23-15.0,9,23,463.31,7,0 days 23:14:21,3.30,2,2.00,100.00
323,48370509,slicelife,2019-03-24 17:55:18+00:00,Delivery,cash,0,16.52,"6,196,987,925.00",27246,WA,98101,2019-03-24,17.00,2019-03-24-17.0,10,23,463.31,7,1 days 02:40:10,-5.50,2,2.00,100.00


In [201]:
# Merging shopszip with original dataset
shopszip_final=shopszip.reset_index()
shopszip_final=shopszip_final.drop_duplicates(subset=['postal_code'], keep='last').drop(columns=['customer','total_order']).set_index('shops_id')
shopszip_final.info()
#shopscustzip_final=shopscustzip.drop(columns=['customer','total_order'])

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3397 entries, 35952 to 18948
Data columns (total 6 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   postal_code             3397 non-null   int64  
 1   shops_per_zip           3397 non-null   int64  
 2   cust_per_zip            3397 non-null   int64  
 3   shops_per_cust_per_zip  3397 non-null   float64
 4   zip_cust_2_more         3005 non-null   float64
 5   zip_pct_cust_2_more     3005 non-null   float64
dtypes: float64(3), int64(3)
memory usage: 185.8 KB


In [202]:
shopszip_final.set_index('postal_code')
test3=test2.merge(shopszip_final,left_on='postal_code',right_on='postal_code') 

In [203]:
# Merging shopsales with original dataset
test4=test3.merge(shopsales,left_on='shops_id',right_on='shops_id')

In [204]:
test4[test4['customer']==6196987925]

,order_id,source,date_purchased,shipping_type,payment_method,promo_value,restaurant_total,customer,shops_id,state,postal_code,dateonly_purchased,houronly_purchased,datehour_purchased,order_enum,total_order,total_sales,bin_orders,time_between_purch,spend_growth,cust_per_shop,shop_cust_2_more,shop_pct_cust_2_more,shops_per_zip,cust_per_zip,shops_per_cust_per_zip,zip_cust_2_more,zip_pct_cust_2_more,total_shop_sales,total_shop_orders,total_shop_cust_base,sales_qunitiles,order_qunitiles
670,47474511,slicelife,2019-03-13 16:33:51+00:00,Delivery,cash,0,20.92,"6,196,987,925.00",27246,WA,98101,2019-03-13,16.00,2019-03-13-16.0,1,23,463.31,7,NaT,nan,2,2.00,100.00,3,13,0.23,10.00,76.92,518.54,23,2,4,4
671,47513940,slicelife,2019-03-14 13:15:11+00:00,Delivery,cash,0,20.92,"6,196,987,925.00",27246,WA,98101,2019-03-14,13.00,2019-03-14-13.0,2,23,463.31,7,0 days 20:41:20,0.00,2,2.00,100.00,3,13,0.23,10.00,76.92,518.54,23,2,4,4
672,47566653,slicelife,2019-03-14 18:44:08+00:00,Delivery,cash,0,18.72,"6,196,987,925.00",27246,WA,98101,2019-03-14,18.00,2019-03-14-18.0,3,23,463.31,7,0 days 05:28:57,-2.20,2,2.00,100.00,3,13,0.23,10.00,76.92,518.54,23,2,4,4
673,47716284,slicelife,2019-03-16 14:25:01+00:00,Delivery,cash,0,20.92,"6,196,987,925.00",27246,WA,98101,2019-03-16,14.00,2019-03-16-14.0,4,23,463.31,7,1 days 19:40:53,2.20,2,2.00,100.00,3,13,0.23,10.00,76.92,518.54,23,2,4,4
674,47947731,slicelife,2019-03-19 17:56:56+00:00,Delivery,cash,0,23.12,"6,196,987,925.00",27246,WA,98101,2019-03-19,17.00,2019-03-19-17.0,5,23,463.31,7,3 days 03:31:55,2.20,2,2.00,100.00,3,13,0.23,10.00,76.92,518.54,23,2,4,4
675,47993268,slicelife,2019-03-20 16:09:30+00:00,Delivery,cash,0,23.12,"6,196,987,925.00",27246,WA,98101,2019-03-20,16.00,2019-03-20-16.0,6,23,463.31,7,0 days 22:12:34,0.00,2,2.00,100.00,3,13,0.23,10.00,76.92,518.54,23,2,4,4
676,48058374,slicelife,2019-03-21 15:29:23+00:00,Delivery,cash,0,19.82,"6,196,987,925.00",27246,WA,98101,2019-03-21,15.00,2019-03-21-15.0,7,23,463.31,7,0 days 23:19:53,-3.30,2,2.00,100.00,3,13,0.23,10.00,76.92,518.54,23,2,4,4
677,48162981,slicelife,2019-03-22 16:00:47+00:00,Delivery,cash,0,18.72,"6,196,987,925.00",27246,WA,98101,2019-03-22,16.00,2019-03-22-16.0,8,23,463.31,7,1 days 00:31:24,-1.10,2,2.00,100.00,3,13,0.23,10.00,76.92,518.54,23,2,4,4
678,48254211,slicelife,2019-03-23 15:15:08+00:00,Delivery,cash,0,22.02,"6,196,987,925.00",27246,WA,98101,2019-03-23,15.00,2019-03-23-15.0,9,23,463.31,7,0 days 23:14:21,3.30,2,2.00,100.00,3,13,0.23,10.00,76.92,518.54,23,2,4,4
679,48370509,slicelife,2019-03-24 17:55:18+00:00,Delivery,cash,0,16.52,"6,196,987,925.00",27246,WA,98101,2019-03-24,17.00,2019-03-24-17.0,10,23,463.31,7,1 days 02:40:10,-5.50,2,2.00,100.00,3,13,0.23,10.00,76.92,518.54,23,2,4,4


In [205]:
## add more data operation statements here if needed

# ***Customer Level Dataset Creation***

In [206]:
test4.head()

,order_id,source,date_purchased,shipping_type,payment_method,promo_value,restaurant_total,customer,shops_id,state,postal_code,dateonly_purchased,houronly_purchased,datehour_purchased,order_enum,total_order,total_sales,bin_orders,time_between_purch,spend_growth,cust_per_shop,shop_cust_2_more,shop_pct_cust_2_more,shops_per_zip,cust_per_zip,shops_per_cust_per_zip,zip_cust_2_more,zip_pct_cust_2_more,total_shop_sales,total_shop_orders,total_shop_cust_base,sales_qunitiles,order_qunitiles
0,47433960,iosapp,2019-03-13 08:02:07+00:00,Delivery,credit,0,15.50,"13,315,067,198.00",24597,MD,21162,2019-03-13,8.00,2019-03-13-8.0,1,2,32.21,1,NaT,nan,2,2.00,100.00,1,2,0.50,2.00,100.00,70.41,4,2,1,2
1,55207974,iosapp,2019-06-22 07:38:20+00:00,Delivery,credit,0,16.71,"13,315,067,198.00",24597,MD,21162,2019-06-22,7.00,2019-06-22-7.0,2,2,32.21,2,100 days 23:36:13,1.21,2,2.00,100.00,1,2,0.50,2.00,100.00,70.41,4,2,1,2
2,59481261,iosapp,2019-08-15 08:58:47+00:00,Pickup,credit,4,13.95,"12,316,161,278.00",24597,MD,21162,2019-08-15,8.00,2019-08-15-8.0,1,2,38.20,1,NaT,nan,2,2.00,100.00,1,2,0.50,2.00,100.00,70.41,4,2,1,2
3,62794419,partner_website,2019-09-24 10:01:02+00:00,Pickup,credit,0,24.25,"12,316,161,278.00",24597,MD,21162,2019-09-24,10.00,2019-09-24-10.0,2,2,38.20,2,40 days 01:02:15,10.30,2,2.00,100.00,1,2,0.50,2.00,100.00,70.41,4,2,1,2
4,47434359,iosapp,2019-03-13 08:11:38+00:00,Delivery,credit,0,24.83,"15,557,129,447.00",64392,NY,12944,2019-03-13,8.00,2019-03-13-8.0,1,5,152.83,1,NaT,nan,3,3.00,100.00,2,5,0.40,6.00,120.00,287.52,12,3,3,4


In [207]:
test4.customer.nunique()

10316

In [208]:
# First Purchase Characteristics

fp=test4[test4['order_enum']==1]

In [240]:
fp.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10316 entries, 0 to 39812
Data columns (total 33 columns):
 #   Column                  Non-Null Count  Dtype              
---  ------                  --------------  -----              
 0   order_id                10316 non-null  int64              
 1   source                  10316 non-null  object             
 2   date_purchased          10316 non-null  datetime64[ns, UTC]
 3   shipping_type           10316 non-null  object             
 4   payment_method          10316 non-null  object             
 5   promo_value             10316 non-null  int64              
 6   restaurant_total        10316 non-null  float64            
 7   customer                10316 non-null  float64            
 8   shops_id                10316 non-null  int64              
 9   state                   10301 non-null  object             
 10  postal_code             10316 non-null  int64              
 11  dateonly_purchased      10316 non-null  o

In [288]:
fp_final=fp[['customer','date_purchased','source','promo_value','shops_id','postal_code','total_order','total_sales','sales_qunitiles','order_qunitiles','zip_pct_cust_2_more','shop_pct_cust_2_more','shops_per_zip', 'shops_per_cust_per_zip']].rename(columns={'date_purchased':'fp_date','source':'fp_source','promo_value':'fp_promo_val','shops_id':'fp_shops_id','postal_code':'fp_postal_code','sales_quintiles':'fp_shop_sales_quintile','order_quintiles':'fp_shop_order_qunitile','zip_pct_cust_2_more':'fp_zip_cust_loyalty_score','shop_pct_cust_2_more':'fp_shop_cust_loyalty_score'})

fp_final['customer_sales_centiles']=pd.qcut(fp_final['total_sales'].rank(method='first'),100,labels=False)
fp_final['customer_order_centiles']=pd.qcut(fp_final['total_order'].rank(method='first'),100,labels=False)

In [296]:
fp_final['end_date']=test['date_purchased'].max()
fp_final['time_since_fp']=((fp_final['end_date']-fp_final['fp_date']).dt.days.astype('int16'))
fp_final['total_order_benchmark']=(fp_final['time_since_fp']*0.5/30).round(0) #6 pizza per year = 0.5 pizza per 30 days
fp_final['total_surplus_orders']=fp_final['total_order']-fp_final['total_order_benchmark']

# make customer total sales and total order centiles, total order benchmark, surplus, shops per zip H/M/L

In [297]:
fp_final.head()

,customer,fp_date,fp_source,fp_promo_val,fp_shops_id,fp_postal_code,total_order,total_sales,sales_qunitiles,order_qunitiles,fp_zip_cust_loyalty_score,fp_shop_cust_loyalty_score,shops_per_zip,shops_per_cust_per_zip,customer_sales_centiles,customer_order_centiles,end_date,time_since_fp,total_order_benchmark,total_surplus_orders
0,"13,315,067,198.00",2019-03-13 08:02:07+00:00,iosapp,0,24597,21162,2,32.21,1,2,100.00,100.00,1,0.50,21,31,2020-05-11 19:13:14+00:00,425,7.00,-5.00
2,"12,316,161,278.00",2019-08-15 08:58:47+00:00,iosapp,4,24597,21162,2,38.20,1,2,100.00,100.00,1,0.50,27,31,2020-05-11 19:13:14+00:00,270,4.00,-2.00
4,"15,557,129,447.00",2019-03-13 08:11:38+00:00,iosapp,0,64392,12944,5,152.83,3,4,120.00,100.00,2,0.40,78,78,2020-05-11 19:13:14+00:00,425,7.00,-2.00
5,"15,557,162,096.00",2019-04-13 15:24:04+00:00,partner_website,0,64392,12944,44,"1,316.66",3,4,120.00,100.00,2,0.40,99,99,2020-05-11 19:13:14+00:00,394,7.00,37.00
35,"21,494,990,414.00",2020-03-23 16:19:50+00:00,partner_website,0,140271,12944,2,55.88,4,4,120.00,100.00,2,0.40,41,31,2020-05-11 19:13:14+00:00,49,1.00,1.00


,customer,fp_date,fp_source,fp_promo_val,fp_shops_id,fp_postal_code,total_order,total_sales,sales_qunitiles,order_qunitiles,fp_zip_cust_loyalty_score,fp_shop_cust_loyalty_score,shops_per_zip,shops_per_cust_per_zip,customer_sales_centiles,end_date
0,"13,315,067,198.00",2019-03-13 08:02:07+00:00,iosapp,0,24597,21162,2,32.21,1,2,100.00,100.00,1,0.50,21,2019-03-31
2,"12,316,161,278.00",2019-08-15 08:58:47+00:00,iosapp,4,24597,21162,2,38.20,1,2,100.00,100.00,1,0.50,27,2019-03-31
4,"15,557,129,447.00",2019-03-13 08:11:38+00:00,iosapp,0,64392,12944,5,152.83,3,4,120.00,100.00,2,0.40,78,2019-03-31
5,"15,557,162,096.00",2019-04-13 15:24:04+00:00,partner_website,0,64392,12944,44,"1,316.66",3,4,120.00,100.00,2,0.40,99,2019-03-31
35,"21,494,990,414.00",2020-03-23 16:19:50+00:00,partner_website,0,140271,12944,2,55.88,4,4,120.00,100.00,2,0.40,41,2019-03-31


In [219]:
#Subsequent Purchase -- Value up to 6th purchase

subs_purchase = test4[(test4['order_enum'] <7)]

In [234]:
subs_purchase_value_final=subs_purchase.pivot(index='customer',columns='order_enum', values='restaurant_total').reset_index().add_prefix('total_spend_')
subs_purchase_value_final.rename(columns={'total_spend_customer':'customer'}).fillna(0)

#subs_purchase_final.drop(columns=['order_enum'])

order_enum,customer,total_spend_1,total_spend_2,total_spend_3,total_spend_4,total_spend_5,total_spend_6
0,"1,920,771,353.00",46.42,0.00,0.00,0.00,0.00,0.00
1,"4,042,653,893.00",30.51,18.36,25.34,15.51,0.00,0.00
2,"4,555,796,885.00",29.78,0.00,0.00,0.00,0.00,0.00
3,"6,033,333,473.00",12.24,0.00,0.00,0.00,0.00,0.00
4,"6,036,185,756.00",44.75,0.00,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...
10311,"51,227,013,587.00",35.70,35.79,42.25,54.54,0.00,0.00
10312,"57,523,677,443.00",12.39,22.29,23.28,20.64,0.00,0.00
10313,"120,495,000,000.00",5.82,0.00,0.00,0.00,0.00,0.00
10314,"217,278,000,000.00",16.84,0.00,0.00,0.00,0.00,0.00


In [298]:
subs_purchase_value_final['spend_growth_2']=(subs_purchase_value_final['total_spend_2']-subs_purchase_value_final['total_spend_1'])*100/subs_purchase_value_final['total_spend_1']
subs_purchase_value_final['spend_growth_3']=(subs_purchase_value_final['total_spend_3']-subs_purchase_value_final['total_spend_2'])*100/subs_purchase_value_final['total_spend_2']
subs_purchase_value_final['spend_growth_4']=(subs_purchase_value_final['total_spend_4']-subs_purchase_value_final['total_spend_3'])*100/subs_purchase_value_final['total_spend_3']
subs_purchase_value_final['spend_growth_5']=(subs_purchase_value_final['total_spend_5']-subs_purchase_value_final['total_spend_4'])*100/subs_purchase_value_final['total_spend_4']
subs_purchase_value_final['spend_growth_6']=(subs_purchase_value_final['total_spend_6']-subs_purchase_value_final['total_spend_5'])*100/subs_purchase_value_final['total_spend_5']

In [323]:
#Subsequent Purchase -- Time up to 6th purchase
subs_purchase_time_final=subs_purchase.pivot(index='customer',columns='order_enum', values='time_between_purch').reset_index().add_prefix('time_to_')

# Converting time between purchases to days
subs_purchase_time_final['time_to_2']=subs_purchase_time_final['time_to_2'].dt.days
subs_purchase_time_final['time_to_3']=subs_purchase_time_final['time_to_3'].dt.days
subs_purchase_time_final['time_to_4']=subs_purchase_time_final['time_to_4'].dt.days
subs_purchase_time_final['time_to_5']=subs_purchase_time_final['time_to_5'].dt.days
subs_purchase_time_final['time_to_6']=subs_purchase_time_final['time_to_6'].dt.days

subs_purchase_time_final.rename(columns={'time_to_customer':'customer'}).drop(columns=['time_to_1'])




order_enum,customer,time_to_2,time_to_3,time_to_4,time_to_5,time_to_6
0,"1,920,771,353.00",nan,nan,nan,nan,nan
1,"4,042,653,893.00",36.00,11.00,2.00,nan,nan
2,"4,555,796,885.00",nan,nan,nan,nan,nan
3,"6,033,333,473.00",nan,nan,nan,nan,nan
4,"6,036,185,756.00",nan,nan,nan,nan,nan
...,...,...,...,...,...,...
10311,"51,227,013,587.00",18.00,24.00,203.00,nan,nan
10312,"57,523,677,443.00",30.00,195.00,17.00,nan,nan
10313,"120,495,000,000.00",nan,nan,nan,nan,nan
10314,"217,278,000,000.00",nan,nan,nan,nan,nan


In [321]:
fp_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10316 entries, 0 to 39812
Data columns (total 20 columns):
 #   Column                      Non-Null Count  Dtype              
---  ------                      --------------  -----              
 0   customer                    10316 non-null  float64            
 1   fp_date                     10316 non-null  datetime64[ns, UTC]
 2   fp_source                   10316 non-null  object             
 3   fp_promo_val                10316 non-null  int64              
 4   fp_shops_id                 10316 non-null  int64              
 5   fp_postal_code              10316 non-null  int64              
 6   total_order                 10316 non-null  int64              
 7   total_sales                 10316 non-null  float64            
 8   sales_qunitiles             10316 non-null  int64              
 9   order_qunitiles             10316 non-null  int64              
 10  fp_zip_cust_loyalty_score   9812 non-null   float64       

In [324]:
subs_purchase_time_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10316 entries, 0 to 10315
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype          
---  ------            --------------  -----          
 0   time_to_customer  10316 non-null  float64        
 1   time_to_1         0 non-null      timedelta64[ns]
 2   time_to_2         7045 non-null   float64        
 3   time_to_3         4306 non-null   float64        
 4   time_to_4         2980 non-null   float64        
 5   time_to_5         2180 non-null   float64        
 6   time_to_6         1707 non-null   float64        
dtypes: float64(6), timedelta64[ns](1)
memory usage: 564.3 KB


In [319]:
subs_purchase_value_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10316 entries, 0 to 10315
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   total_spend_customer  10316 non-null  float64
 1   total_spend_1         10316 non-null  float64
 2   total_spend_2         7047 non-null   float64
 3   total_spend_3         4310 non-null   float64
 4   total_spend_4         2982 non-null   float64
 5   total_spend_5         2182 non-null   float64
 6   total_spend_6         1708 non-null   float64
 7   spend_growth_2        7047 non-null   float64
 8   spend_growth_3        4310 non-null   float64
 9   spend_growth_4        2982 non-null   float64
 10  spend_growth_5        2182 non-null   float64
 11  spend_growth_6        1708 non-null   float64
dtypes: float64(12)
memory usage: 967.2 KB


In [325]:
cust_final_dataset=fp_final.merge(subs_purchase_value_final,left_on='customer',right_on='total_spend_customer') 

In [327]:
cust_final_dataset=cust_final_dataset.merge(subs_purchase_time_final,left_on='customer',right_on='time_to_customer') 

In [328]:
cust_final_dataset.head()

,customer,fp_date,fp_source,fp_promo_val,fp_shops_id,fp_postal_code,total_order,total_sales,sales_qunitiles,order_qunitiles,fp_zip_cust_loyalty_score,fp_shop_cust_loyalty_score,shops_per_zip,shops_per_cust_per_zip,customer_sales_centiles,customer_order_centiles,end_date,time_since_fp,total_order_benchmark,total_surplus_orders,total_spend_customer,total_spend_1,total_spend_2,total_spend_3,total_spend_4,total_spend_5,total_spend_6,spend_growth_2,spend_growth_3,spend_growth_4,spend_growth_5,spend_growth_6,time_to_customer,time_to_1,time_to_2,time_to_3,time_to_4,time_to_5,time_to_6
0,"13,315,067,198.00",2019-03-13 08:02:07+00:00,iosapp,0,24597,21162,2,32.21,1,2,100.00,100.00,1,0.50,21,31,2020-05-11 19:13:14+00:00,425,7.00,-5.00,"13,315,067,198.00",15.50,16.71,nan,nan,nan,nan,7.81,nan,nan,nan,nan,"13,315,067,198.00",NaT,100.00,nan,nan,nan,nan
1,"12,316,161,278.00",2019-08-15 08:58:47+00:00,iosapp,4,24597,21162,2,38.20,1,2,100.00,100.00,1,0.50,27,31,2020-05-11 19:13:14+00:00,270,4.00,-2.00,"12,316,161,278.00",13.95,24.25,nan,nan,nan,nan,73.84,nan,nan,nan,nan,"12,316,161,278.00",NaT,40.00,nan,nan,nan,nan
2,"15,557,129,447.00",2019-03-13 08:11:38+00:00,iosapp,0,64392,12944,5,152.83,3,4,120.00,100.00,2,0.40,78,78,2020-05-11 19:13:14+00:00,425,7.00,-2.00,"15,557,129,447.00",24.83,48.27,40.86,12.26,26.61,nan,94.40,-15.35,-70.00,117.05,nan,"15,557,129,447.00",NaT,48.00,182.00,141.00,10.00,nan
3,"15,557,162,096.00",2019-04-13 15:24:04+00:00,partner_website,0,64392,12944,44,"1,316.66",3,4,120.00,100.00,2,0.40,99,99,2020-05-11 19:13:14+00:00,394,7.00,37.00,"15,557,162,096.00",39.67,21.91,43.98,19.78,21.00,15.53,-44.77,100.73,-55.03,6.17,-26.05,"15,557,162,096.00",NaT,40.00,12.00,35.00,6.00,5.00
4,"21,494,990,414.00",2020-03-23 16:19:50+00:00,partner_website,0,140271,12944,2,55.88,4,4,120.00,100.00,2,0.40,41,31,2020-05-11 19:13:14+00:00,49,1.00,1.00,"21,494,990,414.00",26.86,29.02,nan,nan,nan,nan,8.04,nan,nan,nan,nan,"21,494,990,414.00",NaT,5.00,nan,nan,nan,nan


# ***Exporting Dataset for usage in Tableau***

In [330]:
#test4.to_csv('fin_an_dataset.csv',index=False)
cust_final_dataset.to_csv('cust_final_dataset.csv',index=False)

# ***Define Cohorts Based on First Order***